# Notebook for cleaning `CONDITIONS TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)

In [2]:
sql('select * from conditions limit 5')

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder)


In [3]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'conditions' and
    table_schema = database();
'''
sql(query)
conditions_columns = [c[0] for c in sql(query).values.tolist()]

In [4]:
for col in conditions_columns:
    query = f'''
        alter table conditions
        rename column {col} to {col.lower()}
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [5]:
for col in conditions_columns:
    query = f'''
        select sum({col} is null) as {col}_null from conditions
        having {col}_null > 0;
    '''
    display(sql(query))

,START_null


,STOP_null
0,3811.0


,PATIENT_null


,ENCOUNTER_null


,CODE_null


,DESCRIPTION_null


In [6]:
query = """
ALTER TABLE conditions
MODIFY start DATE,
modify stop date;
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [7]:
for col in conditions_columns:
    query = f'''
        update conditions
        set {col} = lower({col})
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [8]:
sql('select * from conditions limit 5')

,start,stop,patient,encounter,code,description
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,acute viral pharyngitis (disorder)


In [11]:
query = '''
    alter table conditions
    drop column code;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [13]:
sql('select * from conditions limit 5')

,start,stop,patient,encounter,description
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,acute viral pharyngitis (disorder)
